# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 20 2022 9:05AM,253468,12,HH8553,Hush Hush,Released
1,Dec 20 2022 9:05AM,253467,12,HH8558,Hush Hush,Released
2,Dec 20 2022 9:04AM,253466,12,HH8545,Hush Hush,Released
3,Dec 20 2022 9:04AM,253465,12,HH8543,Hush Hush,Released
4,Dec 20 2022 8:59AM,253460,10,0086257411,ISDIN Corporation,Released
5,Dec 20 2022 8:59AM,253460,10,0086257410,ISDIN Corporation,Released
6,Dec 20 2022 8:59AM,253460,10,0086257424,ISDIN Corporation,Released
7,Dec 20 2022 8:59AM,253460,10,0086257422,ISDIN Corporation,Released
8,Dec 20 2022 8:59AM,253460,10,0086257423,ISDIN Corporation,Released
9,Dec 20 2022 8:59AM,253460,10,0086257435,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,253463,Released,2
36,253465,Released,1
37,253466,Released,1
38,253467,Released,1
39,253468,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
253463,2
253465,1
253466,1
253467,1
253468,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
253377,1
253382,1
253383,1
253384,3
253387,3


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
253377,1
253382,1
253383,1
253384,3
253387,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,253377,1
1,253382,1
2,253383,1
3,253384,3
4,253387,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,253377,1
1,253382,1
2,253383,1
3,253384,3
4,253387,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 20 2022 9:05AM,253468,12,Hush Hush
1,Dec 20 2022 9:05AM,253467,12,Hush Hush
2,Dec 20 2022 9:04AM,253466,12,Hush Hush
3,Dec 20 2022 9:04AM,253465,12,Hush Hush
4,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation
42,Dec 20 2022 8:50AM,253462,10,ISDIN Corporation
48,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC"
50,Dec 20 2022 8:46AM,253459,10,"CLINUVEL, Inc."
52,Dec 20 2022 8:46AM,253458,12,Hush Hush
63,Dec 20 2022 8:40AM,253457,16,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 9:05AM,253468,12,Hush Hush,1
1,Dec 20 2022 9:05AM,253467,12,Hush Hush,1
2,Dec 20 2022 9:04AM,253466,12,Hush Hush,1
3,Dec 20 2022 9:04AM,253465,12,Hush Hush,1
4,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38
5,Dec 20 2022 8:50AM,253462,10,ISDIN Corporation,6
6,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC",2
7,Dec 20 2022 8:46AM,253459,10,"CLINUVEL, Inc.",2
8,Dec 20 2022 8:46AM,253458,12,Hush Hush,11
9,Dec 20 2022 8:40AM,253457,16,"Emersa Waterbox, LLC",1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 9:05AM,253468,12,Hush Hush,1
1,Dec 20 2022 9:05AM,253467,12,Hush Hush,1
2,Dec 20 2022 9:04AM,253466,12,Hush Hush,1
3,Dec 20 2022 9:04AM,253465,12,Hush Hush,1
4,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38
5,Dec 20 2022 8:50AM,253462,10,ISDIN Corporation,6
6,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC",2
7,Dec 20 2022 8:46AM,253459,10,"CLINUVEL, Inc.",2
8,Dec 20 2022 8:46AM,253458,12,Hush Hush,11
9,Dec 20 2022 8:40AM,253457,16,"Emersa Waterbox, LLC",1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 9:05AM,253468,12,Hush Hush,1
1,Dec 20 2022 9:05AM,253467,12,Hush Hush,1
2,Dec 20 2022 9:04AM,253466,12,Hush Hush,1
3,Dec 20 2022 9:04AM,253465,12,Hush Hush,1
4,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 9:05AM,253468,12,Hush Hush,1
1,Dec 20 2022 9:05AM,253467,12,Hush Hush,1
2,Dec 20 2022 9:04AM,253466,12,Hush Hush,1
3,Dec 20 2022 9:04AM,253465,12,Hush Hush,1
4,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 9:05AM,253468,12,Hush Hush,1
1,Dec 20 2022 9:05AM,253467,12,Hush Hush,1
2,Dec 20 2022 9:04AM,253466,12,Hush Hush,1
3,Dec 20 2022 9:04AM,253465,12,Hush Hush,1
4,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,4561767,117
12,1267324,15
15,1774010,56
16,253457,1
19,1013728,55
21,1013590,4
22,253444,1


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,4561767,117
1,12,1267324,15
2,15,1774010,56
3,16,253457,1
4,19,1013728,55
5,21,1013590,4
6,22,253444,1


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,117
1,12,15
2,15,56
3,16,1
4,19,55
5,21,4
6,22,1


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,117
1,12,Released,15
2,15,Released,56
3,16,Released,1
4,19,Released,55


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Released,117,15,56,1,55,4,1


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Released,117,15,56,1,55,4,1


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Released,117,15,56,1,55,4,1


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()